In [40]:
%load_ext cudf.pandas

import pandas as pd

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


In [41]:
en_data = pd.read_csv("../data/en_data.csv")
en_data.sample(3, random_state=13)

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
5536,3,0,3,0,1,@arionnee smh iont understand it but fuck it m...
3989,3,0,3,0,1,@MIKEB_IN_THE_D you're still a free world bitc...
12482,3,2,1,0,0,Lmao Lebron's expression says it all RT &#8220...


In [42]:
id_data = pd.read_csv("../data/id_data.csv", encoding="latin1")
id_data.sample(3, random_state=13)

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
10010,"Presiden kelima RI, Megawati Soekarnoputri men...",0,0,0,0,0,0,0,0,0,0,0,0
8642,Kapan punya program begini pak USER ditunggu p...,0,0,0,0,0,0,0,0,0,0,0,0
12472,USER Yg di tawari bukan aseng tapi orang asing...,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
en_data_class_map = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}

en_data["class"] = en_data["class"].map(en_data_class_map)
en_data = en_data[["tweet", "class"]]

en_data.sample(3, random_state=13)

,tweet,class
5536,@arionnee smh iont understand it but fuck it m...,Offensive Language
3989,@MIKEB_IN_THE_D you're still a free world bitc...,Offensive Language
12482,Lmao Lebron's expression says it all RT &#8220...,Hate Speech


In [44]:
id_data["class"] = id_data.apply(
    lambda x: (
        "Hate Speech"
        if x["HS"] == 1
        else "Offensive Language" if x["Abusive"] == 1 else "Neither"
    ),
    axis=1,
)
id_data = id_data[["Tweet", "class"]]
id_data.columns = ["tweet", "class"]

id_data.sample(3, random_state=13)

,tweet,class
10010,"Presiden kelima RI, Megawati Soekarnoputri men...",Neither
8642,Kapan punya program begini pak USER ditunggu p...,Neither
12472,USER Yg di tawari bukan aseng tapi orang asing...,Neither


In [45]:
id_data["lang"] = "id"
en_data["lang"] = "en"

data = pd.concat([id_data, en_data], ignore_index=True)
data.sample(3, random_state=13)

,tweet,class,lang
14034,"#porn,#android,#iphone,#ipad,#sex,#xxx, | #Clo...",Offensive Language,en
20723,Alex Smith wears red and yellow now so quit be...,Neither,en
2004,USER SIRIK AE LU ONTA\xf0\x9f\x98\x83',Hate Speech,id


In [46]:
en_abusive = pd.read_csv("../data/en_abusive.csv")
id_abusive = pd.read_csv("../data/id_abusive.csv")

en_abusive.columns = ["abuses"]
en_abusive["lang"] = "en"

id_abusive.columns = ["abuses"]
id_abusive["lang"] = "id"
stopwords = pd.concat([en_abusive, id_abusive], ignore_index=True)

In [47]:
en_abusive.head(3)

,abuses,lang
0,shit,en
1,fuck,en
2,damn,en


In [48]:
id_abusive.head(3)

,abuses,lang
0,alay,id
1,ampas,id
2,buta,id


In [49]:
stopwords.head(3)

,abuses,lang
0,shit,en
1,fuck,en
2,damn,en


In [50]:
id_slang = pd.read_csv("../data/id_slangword.csv", encoding="latin1", header=None)
id_slang.columns = ["slang", "formal"]
id_slang_map = id_slang.set_index("slang")["formal"].to_dict()
id_slang_map

{'anakjakartaasikasik': 'anak jakarta asyik asyik',
 'pakcikdahtua': 'pak cik sudah tua',
 'pakcikmudalagi': 'pak cik muda lagi',
 't3tapjokowi': 'tetap jokowi',
 '3x': 'tiga kali',
 'aamiin': 'amin',
 'aamiinn': 'amin',
 'aamin': 'amin',
 'aammiin': 'amin',
 'abis': 'habis',
 'abisin': 'habiskan',
 'acau': 'kacau',
 'achok': 'ahok',
 'ad': 'ada',
 'adek': 'adik',
 'adl': 'adalah',
 'adlah': 'adalah',
 'adlh': 'adalah',
 'ado': 'ada',
 'aduhh': 'aduh',
 'aer': 'air',
 'afdol': 'afdal',
 'agamaataualqur': 'agama alquran',
 'agm': 'agama',
 'agma': 'agama',
 'ahaha': 'haha',
 'ahahaha': 'haha',
 'ahehehehe': 'hehe',
 'ahir': 'akhir',
 'ahirnya': 'akhirnya',
 'ahk': 'ahok',
 'ahlamdulillah': 'alhamdulillah',
 'ahli2': 'para ahli',
 'ahlusunnah': 'ahlus sunah',
 'ahmaddani': 'ahmad dhani',
 'aho': 'ahok',
 'ahoax': 'ahok',
 'ahoaxx': 'ahok',
 'ahog': 'ahok',
 'ahokataudjarot': 'ahok djarot',
 'ahokbebanijokowi': 'ahok beban jokowi',
 'ahokbtp': 'ahok basuki tjahaja purnama',
 'ahokditolakw

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory as IdStemmerFactory
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import PorterStemmer
import re

# Initialize stemmers
id_factory = IdStemmerFactory()
id_stemmer = id_factory.create_stemmer()
en_stemmer = PorterStemmer()


# Preprocessing functions
def lowercase(text):
    return text.lower()


def remove_unnecessary_char(text):
    text = re.sub("\n", " ", text)  # Remove '\n'
    text = re.sub("rt", " ", text)  # Remove retweet symbol
    text = re.sub(r"\bRT\b(?!\w)", " ", text)  # Remove standalone 'RT' only
    text = re.sub(r"\bUSER\b", " ", text)  # Remove standalone 'USER'
    text = re.sub(r"\bURL\b", " ", text)  # Remove standalone 'USER'
    text = re.sub(r"\buser\b", " ", text)  # Remove standalone 'USER'
    text = re.sub("((@[^\s]+)|(#[^\s]+))", " ", text)  # Remove @ and #
    text = re.sub("((www\.[^\s]+)|(https?://[^\s]+))", " ", text)  # Remove URLs
    text = re.sub("  +", " ", text)  # Remove extra spaces
    return text


def remove_non_alphanumeric(text):
    text = re.sub("[^0-9a-zA-Z]+", " ", text)
    return text


def remove_encoded_characters(text):
    text = re.sub(r"\\x[0-9a-fA-F]{2}", "", text)
    return text


def normalize_slangword(text, lang):
    if lang == "id":
        return " ".join(
            [
                id_slang_map[word] if word in id_slang_map else word
                for word in text.split()
            ]
        )
    return text


def stemming(text, lang):
    if lang == "id":
        return id_stemmer.stem(text)
    else:  # lang == 'en'
        return " ".join([en_stemmer.stem(word) for word in text.split()])


def expand_contractions(text):
    text = re.sub(r"(\w+)(\d)", r"\1-\1", text)
    text = re.sub(r'(\w+)"', r"\1-\1", text)
    return text


def remove_special_characters(text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    return text


def collapse_multiple_mentions(text):
    text = re.sub(r"(USER\s+)+", "USER ", text).strip()
    text = re.sub(r"(RT\s+)+", "RT ", text).strip()
    return text


def reduce_repeated_characters(text):
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)
    return text


def remove_stopwords(text, lang):
    if lang == "id":
        stops = set(nltk_stopwords.words("indonesian"))
    else:  # lang == 'en'
        stops = set(nltk_stopwords.words("english"))

    abusive_stops = set(stopwords[stopwords["lang"] == lang]["abuses"].tolist())

    all_stops = stops.union(abusive_stops)

    return " ".join([word for word in text.split() if word.lower() not in all_stops])


def preprocess(text, lang):
    """
    Preprocess text with language-specific handling

    Parameters:
    text (str): The input text to preprocess
    lang (str): Language code ('id' for Indonesian, 'en' for English)

    Returns:
    str: Preprocessed text
    """
    text = lowercase(text)
    text = collapse_multiple_mentions(text)
    text = remove_encoded_characters(text)
    text = remove_unnecessary_char(text)
    text = expand_contractions(text)
    text = reduce_repeated_characters(text)
    text = remove_special_characters(text)
    text = remove_stopwords(text, lang)
    text = normalize_slangword(text, lang)

    text = stemming(text, lang)
    return text

In [52]:
from tqdm import tqdm

tqdm.pandas(desc="Preprocessing tweets")

data["cleaned_tweet"] = data.progress_apply(
    lambda x: preprocess(x["tweet"], x["lang"]), axis=1
)

Preprocessing tweets: 100%|██████████| 37952/37952 [38:07<00:00, 16.59it/s]  


In [64]:
data.to_csv("../data/preprocessed_data.csv", index=False)